# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Final Project: Part 2 - Feature Extraction


In any practical machine learning problem, the data preparation and feature extraction stages are the most important and time-consuming. The final project exposes you to a real-world dataset. In this part of the final project, you are responsible to creating features that will be meaningful for prediction. Features are evaluated based on Information Gain, which you implemented in [Assignment 2](../assignment-2/assignment-2.ipynb).

Here is what will work well in this project:

* Extract some sample data, load it in [R](https://www.r-project.org), and do some intial analysis. Feel free to build models there to get a feel for the best features.
* Join the different tables--they are there for a reason. 
* Get creative.
* Read some of the Kaggle competition forums and kernels. 

Here is what will NOT work:

* Do not use only the features as provided in application_train.
* Do not try implementing new learning algorithm in order to generate features. If you find something that works, investigate what features were helpful and add the features. 
* Do not build lookup tables "embeddings" or other things you might have read about but were not covered in class. 
* Do not try to build a kernel matrix on all pairs. Re-evaluate the kernel instead.

In [5]:
require './assignment_lib'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

In [6]:
dir = "/home/dataset"
$dev_db = SQLite3::Database.new "#{dir}/credit_risk_data_dev.db", results_as_hash: true, readonly: true

#<SQLite3::Database:0x00000000036c1bc0 @tracefunc=nil, @authorizer=nil, @encoding=nil, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=true, @type_translation=nil, @readonly=true>

## Question 1.1 (10 Points)

Implement ```create_dataset``` which runs an SQL query on a database and constructs a dataset like those we have used in this course. Add an ```id``` field for the ```SK_ID_CURR``` and store the ```TARGET``` in ```label```. 

If the query is:
```sql
select sk_id_curr, target, ext_source_1 from application_train  where ext_source_1 <> '' order by sk_id_curr;
```

then the result is:

```json
[
    {"label":1,"id":100002,"features":{"ext_source_1":0.08303696739132256}},
    {"label":0,"id":100015,"features":{"ext_source_1":0.7220444501416448}}
]
...
```
Note the features should not include the ID or Label. Feature keys should be lowercase and only contain keys fo which ```key.is_a? String``` returns true.



In [7]:
def create_dataset db, sql
  dataset = []
  db.execute sql do |row|
    # BEGIN YOUR CODE
    r = Hash.new
    r["features"] = Hash.new
    row.each do |key, value|
      if key == "SK_ID_CURR"
        r["id"] = value
      elsif key == "TARGET"
        r["label"] = value
      else
        if key.is_a? String
          r["features"][key.downcase] = value
        end
      end
    end
    dataset << r
    #END YOUR CODE
  end
  return dataset
end

:create_dataset

In [ ]:
def test_11()
  dataset = create_dataset $dev_db, "select sk_id_curr, target, ext_source_1 from application_train where ext_source_1 <> '' 
order by sk_id_curr limit 37"
  assert_equal 37, dataset.size
  assert_true(dataset[0]["features"].has_key? "ext_source_1")
  assert_equal(1, dataset[0]["features"].size)
  assert_equal(100002, dataset[0]["id"])  
  assert_in_delta(0.08303696, dataset[0]["features"]["ext_source_1"], 1e-4)
  assert_equal(1, dataset[0]["label"])    
end

test_11()

## Question 1.1 (20 points)

Copy and revise **your** information gain calculation for numeric and categorical features, from [Assignment 2](../assignment-2/assignment-2.ipynb). Copy the following implementations

* Class Distribution
* Entropy
* Information Gain after splitting
* Information gain for numerical features (fast version)


In [ ]:
def class_distribution dataset
  # BEGIN YOUR CODE
  group = dataset.group_by{|row| row["label"]}
  res = Hash.new {|h,k| h[k] = 0.0}
  
  group.each do |key, value|
    res[key] = value.size.to_f / dataset.size.to_f
  end
  
  return res
  #END YOUR CODE
end

In [ ]:
def entropy dist
  # BEGIN YOUR CODE
  t = dist.values.reduce(0.0, :+)
  h = 0.0
  dist.values.each do |d|
    if d != 0
      h -= (d / t) * Math.log(d / t)
    end
  end
  return h
  #END YOUR CODE
end

In [ ]:
def test_12_1()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  assert_in_delta(0.2686201883261589, h0, 1e-3)
end

test_12_1()

In [ ]:
def information_gain h0, splits
  # BEGIN YOUR CODE
  t = splits.map {|key, value| value.size}.reduce(0.0, :+)
  information_gain = h0
  splits.each do |key, value|
    d = class_distribution(value)
    e = entropy(d)
    information_gain -= (value.size.to_f / t.to_f) * e.to_f
  end
  return information_gain
  #END YOUR CODE
end

In [ ]:
def test_12_2()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  
  splits = dataset.group_by {|row| row["features"]["flag_own_car"]}
  ig = information_gain h0, splits
  assert_in_delta(0.0002206258541794237, ig, 1e-4)
end

test_12_2()

In [ ]:
def find_split_point_numeric x, h0, fname
  # BEGIN YOUR CODE
  sorted = x.sort_by {|row| row["features"][fname] == nil ? 0 : row["features"][fname]}
  vl = Hash.new {|h,k| h[k] = 0.0}
  vr = Hash.new {|h,k| h[k] = 0.0}
  sorted.each do |row|
    vr[row["label"]] += 1.0
  end
  e_min = h0
  t_max = sorted[0]["features"][fname]
  i = 0
  s = sorted.size
  while i < s do
    row = sorted[i]
    vl[row["label"]] += 1.0
    vr[row["label"]] -= 1.0
    if i + 1 < s and row["features"][fname] == sorted[i + 1]["features"][fname]
      i += 1
      next
    end
    e = ((i+1.0)/s)*entropy(vl)+((s-i-1.0)/s)*entropy(vr)
    if e < e_min
      e_min = e
      t_max = sorted[i+1]["features"][fname]
    end
    i += 1
  end
  return [t_max, h0 - e_min]
  #END YOUR CODE
end

In [ ]:
def test_12_3()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  
  t, ig = find_split_point_numeric dataset, h0, "ext_source_1"
  assert_in_delta(0.009751743140812785, ig, 1e-4)
end

test_12_3()

## Question 2.1 (70 Points)

Using whatever external software you want (hosted on your own devices), provide 15+ different features that have information >= 0.005. You may to implement several cells below, so please insert them above the test. 

Features must only be derived from the database but you are free to write whatever SQL queries you want. You may create temporary tables, but the database is read-only.

Pay close attention to the following aspects of feature design:

* Normalization: Z-score, L2, Min-Max, etc.
* Sparsity / missing values
* Frequency: Information is easily fooled by features with many values.
* Joins: Some of the best features in this dataset combine two columns from different tables.
* Transformations: One-hot, Binning, Discretization, Non-linear transformation

In [ ]:
# BEGIN YOUR CODE
def dot x, w
  # BEGIN YOUR CODE
  product = 0.0
  x.each do |key, value|
    if w[key] != nil
      product += value * w[key]
    end
  end
  return product
  #END YOUR CODE
end
def norm w
  # BEGIN YOUR CODE
  return Math.sqrt(dot(w, w))
  #END YOUR CODE
end
#END YOUR CODE

In [ ]:
def hmean data
  res = Hash.new
  count = Hash.new
  data.each do |sample|
    sample["features"].each do |key, value|
      if res[key] == nil
        res[key] = 0.0
      end
      if count[key] == nil
          count[key] = 0
      end
      if value != nil and value.is_a?(String) == false
        res[key] += value
        count[key] += 1
      end
    end
  end
  res.each do |key, value|
    res[key] = value / count[key]
  end
  return res
end
  
def hstd data, mean
  res = Hash.new
  count = Hash.new
  data.each do |sample|
    sample["features"].each do |key, value|
      if res[key] == nil or res[key] == "" or res[key] == ''
        res[key] = 0.0
      end
      if count[key] == nil
        count[key] = 0
      end
      if value != nil and value.is_a?(String) == false
        res[key] += (value - mean[key])**2
        count[key] += 1
      end
    end
  end
  res.each do |key, value|
    if count[key] > 1
      res[key] = Math.sqrt(value / (count[key] - 1))
    else
      res[key] = Math.sqrt(value)
    end
  end
  return res
end

In [ ]:
def z_normalize dataset
  zdataset = dataset.clone
  zdataset = dataset.collect do |r|
    u = r.clone
    u["features"] = r["features"].clone
    u
  end

  # BEGIN YOUR CODE
  mean = hmean(zdataset)
  std = hstd(zdataset, mean)
  zdataset.each do |sample|
    sample["features"].each do |key, value|
      if value.is_a?(String) == false
        if std[key] > 0.0
          sample["features"][key] = (value - mean[key]) / std[key]
        else
          sample["features"][key] = 0.0
        end
      end
    end
  end
  #END YOUR CODE
  return zdataset
end

In [ ]:
def fill_zero dataset
  features = dataset.flat_map {|row| row["features"].keys}.uniq
  numeric_features = features.select {|k| dataset.reject {|row| row["features"][k] == ""}.all? {|row| row["features"].fetch(k, 0.0).is_a? Numeric}}
  categorical_features = features.select {|k| dataset.all? {|row| row["features"].fetch(k, "").is_a? String}}
  dataset.each do |row|
    numeric_features.each do |feature|
      value = row["features"][feature]
      if value == "" or value == ''
        row["features"][feature] = 0.0
      end
    end
    categorical_features.each do |feature|
      value = row["features"][feature]
      if value == "" or value == ''
        row["features"][feature] = "0"
      end
    end
  end
  dataset
end

In [ ]:
def fill_mean dataset
  features = dataset.flat_map {|row| row["features"].keys}.uniq
  numeric_features = features.select {|k| dataset.reject {|row| row["features"][k] == ""}.all? {|row| row["features"].fetch(k, 0.0).is_a? Numeric}}
  categorical_features = features.select {|k| dataset.all? {|row| row["features"].fetch(k, "").is_a? String}}
  mean = hmean dataset 
  dataset.each do |row|
    numeric_features.each do |feature|
      value = row["features"][feature]
      if value == "" or value == ''
        row["features"][feature] = mean[feature]
      end
    end
    categorical_features.each do |feature|
      value = row["features"][feature]
      if value == "" or value == ''
        row["features"][feature] = "0"
      end
    end
  end
  dataset
end

In [ ]:
dataset = create_dataset $dev_db, "SELECT target, sk_id_curr, ext_source_1, ext_source_2, ext_source_3 FROM application_train"
dataset = fill_mean dataset
dataset.each do |row|
  row["features"]["ext_prd"] = row["features"]["ext_source_1"] * row["features"]["ext_source_2"] * row["features"]["ext_source_3"]
  row["features"]["ext_sum"] = row["features"]["ext_source_1"] + row["features"]["ext_source_2"] + row["features"]["ext_source_3"]
  row["features"]["ext_avg"] = row["features"]["ext_sum"] / 3.0
  row["features"]["ext_var"] = (row["features"]["ext_source_1"] - row["features"]["ext_avg"])**2 +
    (row["features"]["ext_source_2"] - row["features"]["ext_avg"])**2 + (row["features"]["ext_source_3"] - row["features"]["ext_avg"])**2
  row["features"]["ext_std"] = row["features"]["ext_var"] / 2.0
end
puts dataset.size

dist = class_distribution dataset
h0 = entropy dist

t, ig = find_split_point_numeric dataset, h0, "ext_source_1"
puts ig

t, ig = find_split_point_numeric dataset, h0, "ext_source_2"
puts ig

t, ig = find_split_point_numeric dataset, h0, "ext_source_3"
puts ig

t, ig = find_split_point_numeric dataset, h0, "ext_prd"
puts ig

t, ig = find_split_point_numeric dataset, h0, "ext_avg"
puts ig

t, ig = find_split_point_numeric dataset, h0, "ext_var"
puts ig

t, ig = find_split_point_numeric dataset, h0, "ext_std"
puts ig

In [ ]:
flag_doc = ""
(2..21).each do |i|
  flag_doc += ", flag_document_#{i.to_s}"
end


dataset = create_dataset $dev_db, "select target, sk_id_curr,
flag_own_car, flag_own_realty, cnt_children, name_contract_type, code_gender, occupation_type
name_type_suite, name_income_type, name_education_type, name_family_status, name_housing_type from application_train"
dataset.each do |row|
  row["features"]["c1"] = 
  row["features"]["flag_own_car"].to_s + row["features"]["flag_own_realty"].to_s + 
  row["features"]["name_income_type"].to_s + row["features"]["name_housing_type"].to_s
  
  row["features"]["c2"] = 
  row["features"]["code_gender"].to_s + row["features"]["cnt_children"].to_s + 
  row["features"]["name_education_type"].to_s + row["features"]["name_family_status"].to_s
  
  row["features"]["c3"] = 
  row["features"]["name_type_suite"].to_s + row["features"]["name_income_type"].to_s +
  row["features"]["name_housing_type"].to_s + row["features"]["name_contract_type"].to_s
  
  row["features"]["c4"] = 
  row["features"]["occupation_type"].to_s + row["features"]["name_income_type"].to_s + 
  row["features"]["name_education_type"].to_s + row["features"]["name_housing_type"].to_s
  
#   row["features"]["flag_doc_num"] = row["features"].map {|key, value| (key.include? "flag") ? value.to_i : 0}.reduce(0, :+).to_s
end
dist = class_distribution dataset
h0 = entropy dist

splits = dataset.group_by {|row| row["features"]["c1"]}
ig = information_gain h0, splits
puts ig

splits = dataset.group_by {|row| row["features"]["c2"]}
ig = information_gain h0, splits
puts ig

splits = dataset.group_by {|row| row["features"]["c3"]}
ig = information_gain h0, splits
puts ig

splits = dataset.group_by {|row| row["features"]["c4"]}
ig = information_gain h0, splits
puts ig

# splits = dataset.group_by {|row| row["features"]["flag_doc_num"]}
# ig = information_gain h0, splits
# puts ig

In [ ]:
dataset = create_dataset $dev_db, "SELECT target, sk_id_curr, region_population_relative, region_rating_client_w_city, REGION_RATING_CLIENT FROM application_train"
dataset = fill_mean dataset
dataset.each do |row|
  row["features"]["region_stat"] = row["features"]["region_population_relative"] - row["features"]["region_rating_client_w_city"]
end
puts dataset.size
dist = class_distribution dataset
h0 = entropy dist

t, ig = find_split_point_numeric dataset, h0, "region_stat"
puts ig

In [ ]:
flag_doc = ""
(2..21).each do |i|
  flag_doc += ", FLAG_DOCUMENT_#{i.to_s}"
end
sql = "SELECT target, sk_id_curr#{flag_doc.to_s} FROM application_train"
dataset = create_dataset $dev_db, sql.to_s

dataset.each do |row|
  row["features"]["flag_doc_sum"] = row["features"].map {|key, value| value}.reduce(0, :+)
end
dist = class_distribution dataset
h0 = entropy dist

t, ig = find_split_point_numeric dataset, h0, "flag_doc_sum"
puts ig

In [ ]:
flag_doc = ""
(2..21).each do |i|
  flag_doc += ", flag_document_#{i.to_s}"
end

amt_credit_bureau = "AMT_REQ_CREDIT_BUREAU_HOUR, AMT_REQ_CREDIT_BUREAU_DAY, 
AMT_REQ_CREDIT_BUREAU_WEEK, AMT_REQ_CREDIT_BUREAU_MON, AMT_REQ_CREDIT_BUREAU_QRT, AMT_REQ_CREDIT_BUREAU_YEAR "
sql = "SELECT target, sk_id_curr#{flag_doc.to_s}, #{amt_credit_bureau} FROM application_train"
dataset = create_dataset $dev_db, sql.to_s
dataset = fill_mean dataset
splitset = Array.new
dataset.each do |row|
  h = Hash.new
  h["id"] = row["id"].clone
  h["label"] = row["label"].clone
  h["features"] = Hash.new
  h["features"]["flag_document_3"] = row["features"]["flag_document_3"]
  h["features"]["flag_doc_sum"] = row["features"].map {|key, value| (key.include? "flag") ? value.to_i : 0}.reduce(0, :+).to_s
#   puts h["features"]["flag_doc_sum"]
  h["features"]["amt_credit_bureau"] = row["features"].map {|key, value| (key.include? "amt") ? value : 0}.reduce(0, :+)
  splitset << h
end
# splitset = z_normalize splitset
# splitset.each do |row|
#   row["features"]["flag_amt_combined"] = row["features"]["flag_doc_sum"] + row["features"]["amt_credit_bureau"]
# end

dist = class_distribution splitset
h0 = entropy dist

splits = dataset.group_by {|row| row["features"]["amt_credit_bureau"]}
ig = information_gain h0, splits
puts ig

# t1, ig1 = find_split_point_numeric splitset, h0, "flag_document_3"
# puts ig1

# t3, ig3 = find_split_point_numeric splitset, h0, "flag_doc_sum"
# puts ig3

# t2, ig2 = find_split_point_numeric splitset, h0, "flag_doc_sum_weighted"
# puts ig2

In [ ]:
dataset = create_dataset $dev_db, "SELECT application_train.sk_id_curr, 
application_train.target, application_train.DAYS_BIRTH, 
application_train.DAYS_EMPLOYED, application_train.DAYS_ID_PUBLISH,
AVG(bureau.DAYS_CREDIT) AS days_credit FROM application_train LEFT JOIN bureau 
ON application_train.sk_id_curr=bureau.sk_id_curr 
GROUP BY application_train.sk_id_curr"
dataset = fill_mean dataset
splitset = Array.new
dataset.each do |row|
  h = Hash.new
  h["id"] = row["id"].clone
  h["label"] = row["label"].clone
  h["features"] = Hash.new
  h["features"]["days_birth_pow"] = row["features"]["days_birth"].to_f**2
  h["features"]["days_credit"] = row["features"]["days_credit"].to_f
  h["features"]["days_employed_pow"] = row["features"]["days_employed"].to_f**2
  h["features"]["days_id_publish"] = row["features"]["days_id_publish"].to_f
  
  h["features"]["days_compound"] = h["features"]["days_birth_pow"] * h["features"]["days_employed_pow"] * 
  h["features"]["days_credit"] * h["features"]["days_id_publish"]
  splitset << h
end
dist = class_distribution splitset

h0 = entropy dist

t, ig = find_split_point_numeric splitset, h0, "days_compound"
puts splitset.size
puts ig

In [ ]:
dataset = create_dataset $dev_db, "SELECT application_train.sk_id_curr, application_train.target, 
application_train.amt_goods_price, application_train.amt_income_total, 
application_train.amt_credit, application_train.AMT_ANNUITY AS at_amt_annuity, 
AVG(previous_application.NAME_CONTRACT_TYPE) AS prev_name_contract_type, 
AVG(previous_application.AMT_APPLICATION) AS prev_amt_application, 
AVG(previous_application.AMT_DOWN_PAYMENT) AS prev_amt_down_payment,
AVG(previous_application.CODE_REJECT_REASON) AS prev_code_reject_reason,
AVG(previous_application.RATE_INTEREST_PRIMARY) AS rate_interest_primary, 
AVG(previous_application.RATE_INTEREST_PRIVILEGED) AS rate_interest_privileged
FROM application_train LEFT JOIN previous_application 
ON application_train.sk_id_curr=previous_application.sk_id_curr 
GROUP BY application_train.sk_id_curr"
dataset = fill_mean dataset
splitset = Array.new
dataset.each do |row|
  h = Hash.new
  h["id"] = row["id"].clone
  h["label"] = row["label"].clone
  h["features"] = Hash.new
  h["features"]["rate_interest_primary"] = row["features"]["rate_interest_primary"]
  h["features"]["rate_interest_privileged"] = row["features"]["rate_interest_privileged"]
  h["features"]["prev_name_contract_type"] = row["features"]["prev_name_contract_type"]
  h["features"]["prev_amt_application"] = row["features"]["prev_amt_application"]
  h["features"]["prev_amt_down_payment"] = row["features"]["prev_amt_down_payment"]
  h["features"]["prev_code_reject_reason"] = row["features"]["prev_code_reject_reason"]
  
  h["features"]["compound"] = h["features"]["rate_interest_primary"]**2
  splitset << h
end

dist = class_distribution splitset

h0 = entropy dist

t, ig = find_split_point_numeric splitset, h0, "compound"
puts splitset.size
puts ig

splits = splitset.group_by {|row| row["features"]["prev_amt_application"]}
ig = information_gain h0, splits
puts ig

In [ ]:
dataset = create_dataset $dev_db, "SELECT application_train.sk_id_curr, application_train.target, 
application_train.AMT_ANNUITY, application_train.AMT_GOODS_PRICE, application_train.AMT_CREDIT, application_train.amt_income_total, 
AVG(previous_application.AMT_ANNUITY) AS prev_amt_annuity, AVG(previous_application.AMT_CREDIT) AS prev_amt_credit, 
AVG(previous_application.AMT_GOODS_PRICE) AS prev_amt_goods_price, AVG(previous_application.RATE_INTEREST_PRIMARY), 
AVG(previous_application.RATE_INTEREST_PRIVILEGED),
AVG(bureau.AMT_ANNUITY) AS bu_amt_annuity, AVG(bureau.AMT_CREDIT_SUM) AS bu_amt_credit
FROM application_train LEFT JOIN previous_application 
ON application_train.sk_id_curr=previous_application.sk_id_curr 
LEFT JOIN bureau ON application_train.sk_id_curr=bureau.sk_id_curr
GROUP BY application_train.sk_id_curr"
dataset = fill_mean dataset
splitset = Array.new
dataset.each do |row|
  h = Hash.new
  h["id"] = row["id"].clone
  h["label"] = row["label"].clone
  h["features"] = Hash.new
  h["features"]["prev_amt_annuity"] = row["features"]["prev_amt_annuity"]
  h["features"]["prev_amt_credit"] = row["features"]["prev_amt_credit"]
  h["features"]["prev_amt_goods_price"] = row["features"]["prev_amt_goods_price"]
  
  h["features"]["amt_annuity"] = row["features"]["amt_annuity"]
  h["features"]["amt_credit"] = row["features"]["amt_credit"]
  h["features"]["amt_goods_price"] = row["features"]["amt_goods_price"]
  
  h["features"]["bu_amt_annuity"] = row["features"]["bu_amt_annuity"]
  h["features"]["bu_amt_credit"] = row["features"]["bu_amt_credit"]
  if h["features"]["prev_amt_annuity"] == 0.0
    h["features"]["prev_credit_to_annuity"] = h["features"]["prev_amt_credit"]
  else
    h["features"]["prev_credit_to_annuity"] = h["features"]["prev_amt_credit"] / h["features"]["prev_amt_annuity"]
  end
  if h["features"]["bu_amt_annuity"] == 0.0
    h["features"]["bu_credit_to_annuity"] = h["features"]["bu_amt_credit"]
  else
    h["features"]["bu_credit_to_annuity"] = h["features"]["bu_amt_credit"] / h["features"]["bu_amt_annuity"]
  end
  if h["features"]["amt_annuity"] == 0.0
    h["features"]["credit_to_annuity"] = h["features"]["amt_credit"]
  else
    h["features"]["credit_to_annuity"] = h["features"]["amt_credit"] / h["features"]["amt_annuity"]
  end
#   h["features"]["credit_sum"] = h["features"]["amt_credit"] + h["features"]["prev_amt_credit"] + h["features"]["bu_amt_credit"]
#   h["features"]["annuity_sum"] = h["features"]["amt_annuity"] + h["features"]["prev_amt_annuity"] + h["features"]["bu_amt_annuity"]
#   h["features"]["goods_price_sum"] = h["features"]["amt_goods_price"] + h["features"]["prev_amt_goods_price"]

  h["features"]["credit_minus_price"] = h["features"]["amt_credit"] - h["features"]["amt_goods_price"]
  h["features"]["prev_credit_minus_price"] = h["features"]["prev_amt_credit"] - h["features"]["prev_amt_goods_price"]
  h["features"]["credit_minus_price_compound"] = 
  (h["features"]["credit_minus_price"] + h["features"]["prev_credit_minus_price"])
  
  h["features"]["compound_credit_to_annuity"] =
  Math.log10(h["features"]["bu_credit_to_annuity"]**4 * h["features"]["credit_to_annuity"])
  
  splitset << h
end

dist = class_distribution splitset

h0 = entropy dist

t, ig = find_split_point_numeric splitset, h0, "credit_minus_price_compound"
puts splitset.size
puts ig

In [ ]:
dataset = create_dataset $dev_db, "SELECT TARGET, SK_ID_CURR AS COUNT FROM application_train WHERE SK_ID_CURR IN (SELECT SK_ID_CURR FROM previous_application GROUP BY SK_ID_CURR) GROUP BY SK_ID_CURR"
dataset.each do |sample|
  puts sample
end

In [ ]:
#Add extra cells as needed

In [ ]:
def extract_features db
  dataset = create_dataset db, "SELECT
  application_train.sk_id_curr,
  application_train.target,
  application_train.ext_source_1,
  application_train.ext_source_2,
  application_train.ext_source_3,
  application_train.flag_own_car,
  application_train.flag_own_realty,
  application_train.cnt_children,
  application_train.name_contract_type,
  application_train.code_gender,
  application_train.name_type_suite,
  application_train.name_income_type,
  application_train.name_education_type,
  application_train.name_family_status,
  application_train.name_housing_type,
  application_train.occupation_type,
  application_train.AMT_ANNUITY,
  application_train.AMT_GOODS_PRICE,
  application_train.AMT_CREDIT,
  application_train.DAYS_BIRTH,
  application_train.DAYS_EMPLOYED,
  application_train.DAYS_ID_PUBLISH,
  AVG(previous_application.AMT_ANNUITY) AS prev_amt_annuity,
  AVG(previous_application.AMT_CREDIT) AS prev_amt_credit,
  AVG(previous_application.AMT_GOODS_PRICE) AS prev_amt_goods_price,
  AVG(bureau.AMT_ANNUITY) AS bu_amt_annuity,
  AVG(bureau.AMT_CREDIT_SUM_DEBT) AS amt_credit_sum_debt,
  AVG(bureau.DAYS_CREDIT) AS days_credit,
  AVG(bureau.AMT_CREDIT_SUM) AS amt_credit_sum,
  AVG(installments_payments.AMT_PAYMENT) AS amt_payment,
  AVG(installments_payments.DAYS_INSTALMENT) AS days_instalment
  FROM
  application_train
  LEFT JOIN bureau ON application_train.sk_id_curr = bureau.sk_id_curr
  LEFT JOIN previous_application ON application_train.sk_id_curr = previous_application.sk_id_curr
  LEFT JOIN installments_payments ON previous_application.sk_id_prev = installments_payments.sk_id_prev
  GROUP BY
  application_train.sk_id_curr"
  dataset = fill_mean dataset
  splitset = Array.new
  
#   credit_to_annuity_max = dataset.map {|row| row["features"]["bu_amt_annuity"].to_f == 0.0 ? row["features"]["amt_credit_sum"].to_f : (row["features"]["amt_credit_sum"].to_f / row["features"]["bu_amt_annuity"].to_f)}.max
#   credit_to_annuity_min = dataset.map {|row| row["features"]["bu_amt_annuity"].to_f == 0.0 ? row["features"]["amt_credit_sum"].to_f : (row["features"]["amt_credit_sum"].to_f / row["features"]["bu_amt_annuity"].to_f)}.min
    
  
  dataset.each do |row|
    h = Hash.new
    h["features"] = Hash.new
    h["id"] = row["id"].clone
    h["label"] = row["label"].clone
    
    h["features"]["ext_source_2"] = row["features"]["ext_source_2"]
    h["features"]["ext_source_3"] = row["features"]["ext_source_3"]
    h["features"]["ext_prd"] = 
    row["features"]["ext_source_1"] * row["features"]["ext_source_2"] * row["features"]["ext_source_3"]
    h["features"]["ext_avg"] = 
    (row["features"]["ext_source_1"] + row["features"]["ext_source_2"] + row["features"]["ext_source_3"]) / 3.0
    
    h["features"]["c_1"] = 
    row["features"]["flag_own_car"].to_s + row["features"]["flag_own_realty"].to_s + 
    row["features"]["name_income_type"].to_s + row["features"]["name_housing_type"].to_s
    
    h["features"]["c_2"] = 
    row["features"]["code_gender"].to_s + row["features"]["cnt_children"].to_s + 
    row["features"]["name_education_type"].to_s + row["features"]["name_family_status"].to_s

    h["features"]["c_3"] = 
    row["features"]["name_type_suite"].to_s + row["features"]["name_income_type"].to_s +
    row["features"]["name_housing_type"].to_s + row["features"]["name_contract_type"].to_s
    
    h["features"]["c_4"] = 
    row["features"]["occupation_type"].to_s + row["features"]["name_education_type"].to_s + 
    row["features"]["name_housing_type"].to_s
    
    h["features"]["days_compound"] = 
    (row["features"]["days_birth"].to_f)**2 * (row["features"]["days_employed"].to_f)**2 * 
    (row["features"]["days_credit"].to_f * row["features"]["days_id_publish"].to_f) / 10**15
    
    h["features"]["days_birth_ext"] = 
    (row["features"]["days_birth"].to_f)**2 * 
    row["features"]["ext_source_2"].to_f / 10**5
    
    h["features"]["days_credit_ext"] = 
    (row["features"]["days_credit"].to_f)**2 * 
    row["features"]["ext_source_2"].to_f / 10**5
    
    amt_annuity = row["features"]["amt_annuity"].to_f
    amt_credit = row["features"]["amt_credit"].to_f
    bu_amt_annuity = row["features"]["bu_amt_annuity"].to_f
    bu_amt_credit = row["features"]["amt_credit_sum"].to_f
    amt_credit_sum_debt = row["features"]["amt_credit_sum_debt"].to_f
    amt_credit_sum = row["features"]["amt_credit_sum"].to_f
    amt_goods_price = row["features"]["amt_goods_price"].to_f
    prev_amt_credit = row["features"]["prev_amt_credit"].to_f
    prev_amt_goods_price = row["features"]["prev_amt_goods_price"].to_f
    amt_payment = row["features"]["amt_payment"].to_f
    days_instalment = row["features"]["days_instalment"].to_f
    
    credit_to_annuity = 0.0
    if amt_annuity == 0.0
      credit_to_annuity = amt_credit
    else
      credit_to_annuity = amt_credit / amt_annuity
    end
    
    bu_credit_to_annuity = 0.0
    if bu_amt_annuity == 0.0
      bu_credit_to_annuity = bu_amt_credit
    else
      bu_credit_to_annuity = bu_amt_credit / bu_amt_annuity
    end
    
    if amt_credit_sum == 0.0
      h["features"]["debt_to_credit"] = (amt_credit_sum_debt**3) + h["features"]["ext_source_2"]
    else
      h["features"]["debt_to_credit"] = 
      ((amt_credit_sum_debt / amt_credit_sum)**3) + h["features"]["ext_source_2"]
    end
    
    credit_minus_price = amt_credit - amt_goods_price
    prev_credit_minus_price = prev_amt_credit - prev_amt_goods_price
    h["features"]["credit_minus_price_ext"] = 
    (((credit_minus_price + prev_credit_minus_price)) / row["features"]["ext_source_2"])
    
    payment_time = (amt_payment * days_instalment)
    h["features"]["payment_time_ext"] = 
    (-payment_time * h["features"]["ext_source_2"])
    
    h["features"]["credit_to_annuity_ext"] = 
    (credit_to_annuity * bu_credit_to_annuity * h["features"]["ext_source_3"]**3)

    splitset << h
  end
  splitset = z_normalize splitset 
  return splitset
end

In [ ]:
dataset = extract_features($dev_db)
dist = class_distribution dataset
h0 = entropy dist
features = dataset.flat_map {|row| row["features"].keys}.uniq
puts dataset.size
numeric_features = features.select {|k| dataset.reject {|row| row["features"][k] == ""}.all? {|row| row["features"].fetch(k, 0.0).is_a? Numeric}}
categorical_features = features.select {|k| dataset.all? {|row| row["features"].fetch(k, "").is_a? String}}
numeric_features.each do |feature|
  t, ig = find_split_point_numeric dataset, h0, feature
  puts feature
  puts ig
  puts "-----------------"
end
categorical_features.each do |feature|
  splits = dataset.group_by {|row| row["features"][feature]}
  ig = information_gain h0, splits
  puts splits.size
  puts feature
  puts ig
  puts "-----------------"
end

In [ ]:
extracted_dataset = extract_features($dev_db)
extracted_dataset[0]

In [ ]:
assert_not_nil extracted_dataset
assert_equal 15334, extracted_dataset.size
assert_true(extracted_dataset.all? {|row| row["features"].size >= 8}, "At least 6 non-zero features per row")
assert_true(extracted_dataset.flat_map {|row| row["features"].keys}.uniq.size >= 15,  "At least 15 features")

In [ ]:
assert_equal 15334, extracted_dataset.collect {|row| row["id"]}.uniq.size
assert_equal 2, extracted_dataset.collect {|row| row["label"]}.uniq.size

h0 = entropy(class_distribution(extracted_dataset))
assert_in_delta(0.2797684909805576, h0, 1e-3)

In [ ]:
features = extracted_dataset.flat_map {|row| row["features"].keys}.uniq
numeric_features = features.select {|k| extracted_dataset.reject {|row| row["features"][k] == ""}.all? {|row| row["features"].fetch(k, 0.0).is_a? Numeric}}

assert_true(numeric_features.size >= 4, "At least 4 numeric features")
def test_ig_numeric extracted_dataset, h0, test_feature1
  t, ig = find_split_point_numeric extracted_dataset, h0, test_feature1
  assert_true(ig >= 0.005, "Expected information gain for '#{test_feature1}' > 0.005")
  return test_feature1
end

test_ig_numeric extracted_dataset, h0, numeric_features[0]

In [ ]:
test_ig_numeric extracted_dataset, h0, numeric_features[1]

In [ ]:
test_ig_numeric extracted_dataset, h0, numeric_features[2]

In [ ]:
3.upto(numeric_features.size - 1) do |i|
  test_ig_numeric extracted_dataset, h0, numeric_features[i]
end

In [ ]:
categorical_features = features.select {|k| extracted_dataset.all? {|row| row["features"].fetch(k, "").is_a? String}}

assert_true(categorical_features.size >= 4, "At least 4 categorical features")

def test_ig_categorical extracted_dataset, h0, test_feature1
  splits = extracted_dataset.group_by {|row| row["features"][test_feature1]}
  ig = information_gain h0, splits
  puts ig
  assert_true(ig >= 0.005, "Expected information gain for '#{test_feature1}' > 0.005")
  return test_feature1
end

test_ig_categorical extracted_dataset, h0, categorical_features[0]

In [ ]:
test_ig_categorical extracted_dataset, h0, categorical_features[1]

In [ ]:
test_ig_categorical extracted_dataset, h0, categorical_features[2]

In [ ]:
3.upto(categorical_features.size - 1) do |i|
  test_ig_categorical extracted_dataset, h0, categorical_features[i]
end